# Open Alex Extraction and Matching with .search()

The goal of this Notebook is look up the PhD students (Authors) contained in the [cleaned](clean_data.ipynb) NARCIS dataset, and
1. Confirm they can be found in OpenAlex
2. Confirm their affiliation in NARCIS matches the one in OpenAlex
2. Confirm they wrote the associated PhD Thesis
3. Per author, look up all the contributors (i.e. potential first supervisors) that are listen in the NARCIS dataset and
    a. Find all authors that have worked for the same organization at the time the PhD thesis was published (within a 1 year window)
    b. xxx


The previous version of this notebook written by a Bachelor student was using the `.search_filter()` method of `pyalex`, which does not search alternate spellings of the specified name. In this notebook we are using `search_filter()`, which does not have that problem. See the example code [here](search_parameter_vs_search_filter.ipynb).

## 1. Setup

In [ ]:
#from pyalex import Works, Authors, Sources, Institutions, Topics, Publishers, Funders, Concepts
from pyalex import config # to set email_address
import pandas as pd
from sentence_transformers import SentenceTransformer
from os import path
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from src.unabbreviate_institutions import unabbreviate_institutions
from src.open_alex_helpers import AuthorRelations, find_phd_and_supervisors_in_row, get_supervisors_openalex_ids
from src.dataset_config_helpers import read_config, load_dataset
from src.api_cache_helpers import initialize_request_cache
from src.plotters import PhDMatchPlotter, ContributorMatchPlotter

# Initialize tqdm for progress bars
tqdm.pandas()

# Install the cache before any API calls are made.
# This will cache every API call to Open Alex and if a cached version of the call is available,
# it will be preferred over making a new API call.
initialize_request_cache()

Notebook settings

In [ ]:
# Automatically reloads any modules that are imported, 
# so that any changes made to the module files are reflected # without needing to restart the Jupyter kernel.
# load autoreload module
%load_ext autoreload
# mode 1 reloads only when an import statement is called. For production
# mode 2 reloads before execution of every cell
%autoreload 2

# limit the number of rows that are shown with printing data frames
pd.set_option('display.max_rows', 5)

Set contact email address to get to use the [polite pool](https://docs.openalex.org/how-to-use-the-api/rate-limits-and-authentication#the-polite-pool). Also, if you are on a premium plan, you can access the higher usage limit by using the associated email address.

In [ ]:
# Get contact email address from file
email_file_path = 'contact_email.txt'

if path.isfile(email_file_path):
    with open(email_file_path, 'r') as file:
        email_address = file.read().strip()

    # Assign the email address to the pyalex configuration
    config.email = email_address

config.email

Configure number of retries and backoff factor
Pyalex is using [urllib3.util.Retry](https://urllib3.readthedocs.io/en/stable/reference/urllib3.util.html) for retrying.

In [ ]:
config.max_retries = 7
config.retry_backoff_factor = 2 # conservative backoff

## 2. Load datasets

### 2.1 Cleaned processed NARCIS dataset

In [ ]:
dataset_path = 'data/cleaned/pubs_with_domain.csv'
output_filename = 'data/output/author_relations.csv'

config = read_config('dataset_config.yaml')

# Rank after which we cut of contributors.
min_rank_contrib = config['min_rank_contrib'] or None
if min_rank_contrib:
    print(f"Considering the first {min_rank_contrib} contributors per PhD candidate.")
else:
    print(f"Considering all contributors.")

pubs_df = load_dataset(dataset_path=dataset_path, config=config)

pubs_df.pop('integer_id')

pubs_df

In [ ]:
# replace institution abbreviation with names that can be found in OpenAlex
# drop exact duplicates after this step
pubs_unabbrev_df = unabbreviate_institutions(pubs_df, 'institution').drop_duplicates()
pubs_unabbrev_df

### 2.2 Handling several contributors per PhD

PhD candidates with 4 or more supervisors (for information).

In [ ]:
contributor_cols = [f'contributor_{i}' for i in range(1, 11)]

# Count non-missing contributor entries per row
pubs_unabbrev_df['contributor_count'] = pubs_unabbrev_df[contributor_cols].notna().sum(axis=1)

# Reorder columns to place contributor_count after institution
cols = list(pubs_unabbrev_df.columns)
if 'institution' in cols and 'contributor_count' in cols:
    cols.remove('contributor_count')
    institution_index = cols.index('institution')
    cols.insert(institution_index + 1, 'contributor_count')
    pubs_unabbrev_df = pubs_unabbrev_df[cols]

# Filter and sort
pubs_more_than_n_df = (
    pubs_unabbrev_df[pubs_unabbrev_df['contributor_count'] >= 4]
    .sort_values(by=['institution', 'contributor_count'], ascending=[True, True])
    .copy()
)

print(f"There are {pubs_more_than_n_df.shape[0]} PhD candidates with 4 or more supervisors")

pubs_more_than_n_df

Remove lower rank contributors, c.f. [#49](https://github.com/StefKirsch/clean-and-enrich-phd-supervisor-data/issues/49).

In [ ]:
pubs_high_contrib_df = pubs_unabbrev_df.copy()

# Get low ranking contributor columns 
if min_rank_contrib:
    low_rank_contrib_cols = [
        col for col in contributor_cols 
        if int(col.split('_')[1]) > min_rank_contrib
    ]

    pubs_high_contrib_df[low_rank_contrib_cols] = pd.NA

pubs_high_contrib_df


### 2.3 Priority supervisor list from ResponsibleSupervision pilot

This dataset was created during the Responsible Supervision pilot project, see [here](https://github.com/tamarinde/ResponsibleSupervision/tree/main/Pilot-responsible-supervision).

In [ ]:
repo_url = "https://github.com/tamarinde/ResponsibleSupervision/tree/main/Pilot-responsible-supervision/data/spreadsheets"
csv_path = "data/output/sups_pilot.csv"

try:
    # Attempt to read the supervisors in the pilot dataset from csv_path
    # If it fails, we get them again from GitHub
    supervisors_in_pilot_dataset = get_supervisors_openalex_ids(repo_url, csv_path)
    print("Unique Supervisors with OpenAlex IDs:")
    print(supervisors_in_pilot_dataset)
except Exception as e:
    print(f"An error occurred: {e}")

## 3. Extraction

Load the pre-trained SPECTER model by allenai (designed for scientific documents). We pre-load the model here, so that we don't need to do that per class instance.

Citation information can be found here: https://github.com/allenai/specter

In [ ]:
model = SentenceTransformer("allenai-specter")

In [ ]:
# set the dict to overwrite the default class attribute specified in src/open_alex_helpers.py
AuthorRelations.supervisors_in_pilot_dataset = supervisors_in_pilot_dataset

# Apply the function to each row with a constant, preloaded model
extraction_series = pubs_high_contrib_df.progress_apply(
    lambda row: find_phd_and_supervisors_in_row(row, model),
    axis=1
    )

# Concatenate all DataFrames into one
extraction_df = pd.concat(list(extraction_series), ignore_index=True)

extraction_df.to_csv(output_filename, index=False)

extraction_df

### Handle duplicate PhDs

In [ ]:
dups = extraction_df[extraction_df.duplicated(subset=['phd_name', 'phd_id'], keep=False)].sort_values(by='phd_name')

dups

## 4. Analysis and Visualization

Load the extraction dataset from file in case we didn't run the extraction

In [ ]:
if 'extraction_df' not in locals() and 'extraction_df' not in globals():
    file_path = output_filename
    
    # Check if the file exists
    if path.exists(file_path):
        extraction_df = pd.read_csv(file_path)
        print(f"Read `extraction_df` from {file_path}")
    else:
        raise FileNotFoundError(f"File not found: {file_path}")
    
extraction_df

In [ ]:
print(f"We managed to find contributors with {extraction_df['n_shared_inst_grad'].sum()} shared institutions and {extraction_df['n_shared_pubs'].sum()} shared publications!")

Get PhDs that we could not find in OpenAlex.

In [ ]:
# Step 1: Filter extraction_df for rows with phd_id = NaN
extraction_none_df = extraction_df.query("phd_id != phd_id")

# Step 2: Filter pubs_unabbrev_df for matching phd_names; then sort and export
pubs_phd_not_confirmed_df = (
    pubs_unabbrev_df
    .query("phd_name in @extraction_none_df.phd_name")
    .sort_values(by=["year", "institution"])   # sort by multiple columns
)

# Export to CSV without the DataFrame index
pubs_phd_not_confirmed_df.to_csv("data/output/phds_not_confirmed.csv", index=False)

pubs_phd_not_confirmed_df

In [ ]:
plotter = PhDMatchPlotter(extraction_df)
ax = plotter.plot()
plt.show()

In [ ]:
plotter = ContributorMatchPlotter(extraction_df)
ax = plotter.plot()
plt.show()